In [4]:
from celery import Celery
from flask import Flask
def make_celery(app):
    celery = Celery(app.import_name, broker=app.config['CELERY_BROKER_URL'])
    celery.conf.update(app.config)
    TaskBase = celery.Task
    class ContextTask(TaskBase):
        abstract = True
        def __call__(self, *args, **kwargs):
            with app.app_context():
                return TaskBase.__call__(self, *args, **kwargs)
    celery.Task = ContextTask
    return celery

In [7]:
app = Flask(__name__)
app.config.update(
    CELERY_BROKER_URL='0.0.0.0',
    CELERY_RESULT_BACKEND='0.0.0.0'
)
celery = make_celery(app)


@celery.task()
def add_together(a, b):
    return a + b

In [8]:
result = add_together.delay(23, 42)
result.wait()

ImportError: No module named '0'